### Test Mainterms with No Levels
* This script tests all mainterms that do not have levels

In [1]:
import json
from pprint import pprint # Used for debugging

# Creating memory allocations to PSC Index, PCS Tables, PCS Definitions json files
with open("Data/JSON/PCS_Index.json") as index_file:
    index = json.load(index_file)
with open("Data/JSON/PCS_Tables.json") as table_file:
    tables = json.load(table_file)
with open("Data/JSON/PCS_Definitions.json") as definitions_file:
    defs = json.load(definitions_file)

### Start class definitions

In [2]:
class Parser:
    def __init__(self):
        self.term_found_flag = False # Flag for ensuring mainterm found. Used for debugging.
        self.pcs_component = False # Flag to let output function know if need to account for PCS Table component.

    def execute_tree(self, mainterm, single_level_check=False):
        """
        Args:
            Mainterm -> Mainterm object in focus.
            single_level_check -> Bool value signaling to check for single level.
        Returns:
            None.
        """
        # This function acts as the 'go-to' execution funtion for this script
        # All "mainterm" objects are passed to this function
        # This function checks for levels, then executes functions built to handle the mainterm's structure
        levels = self.check_for_levels_in_mainterm(mainterm)
        # Check for levels to see which type of execute function is needed
        if levels == "level_1" and single_level_check == True:
            print("--------EXECUTE SINGLE LEVEL--------")
            # Used to execute mainterms that have multiple subterms "_level" key equal to "1" 
            # Does not include mainterms with subterms that have "_level" equal to "2" or greater
            single_level_parser.execute_single_level(mainterm)
        elif not levels:
            print("--------PARENT EXECUTE--------")
            # Used to execute mainterms that do not have levels
            mainterm_parser.parent_execute(mainterm)
        else:
            print("--------PROGRESS THROUGH LEVELS--------")
            # Used to execute mainterms with subterms that have "_level" of 2 or greater
            self.progress_through_levels(mainterm)
    
    def progress_through_levels(self, mainterm):
        """
        Args
            mainterm -> Mainterm object in focus.
        Returns:
            None.
        """
        # This function handles "mainterm" objects that contain more than 1 level
        # It iterates through levels until a final code or term is found
        # This function executes queries using sub-function Mainterm_Parser().progress_through_levels_execute()
        level_flag = True
        while level_flag:
            # Returns all "title" values from the next level of subterms
            new_level_terms = self.get_next_level_title_values(mainterm)
            if new_level_terms == None:
                # If no next level choices, execute with final subterm
                mainterm_parser.progress_through_levels_execute(mainterm)
                level_flag = False
            elif len(new_level_terms) == 1:
                # Find new_level_term. Auto select the first term from the new_level_terms list because only one term available
                new_level_term = new_level_terms[0]
                print(f"Automatically choosing term, '{new_level_term}', because it's the only choice available.")
                # Re-query mainterm object to find subterm that matches our new_level_term
                # Start another while loop with updated mainterm object
                mainterm = self.find_matching_mainterm_to_user_input(mainterm, new_level_term)
            else:
                ### ADDED ###
                # Indicate term note being tested
                # Doing this because We are testing terms with only one level
                global test_flag
                test_flag = "No Test"
                break
                
                """
                Add functionality here to test mainterms with levels
                If the code executes here, that means there are choices for the user to make
                Our test needs to be able to start from the beginning and test each possible combination of answers
                
                1) capture mainterm object 
                2) capture all possible combinations of terms, probably list of lists of each term
                3) test for each possible combination of terms
                """
                
                """
                # If more than one next level choice, ask user to choose
                print(new_level_terms)
                user_input = input("Choose term : ")
                # If user's choice does not match a choice in new_level_terms
                if user_input not in new_level_terms:
                    user_input = self.handle_bad_user_query(user_input, new_level_terms)
                # Find new mainterm object based on user choice above
                mainterm = self.find_matching_mainterm_to_user_input(mainterm, user_input)
                # Check for levels on new mainterm
                levels = self.check_for_levels_in_mainterm(mainterm)
                if not levels:
                    # If no more levels, execute with final mainterm
                    # If more levels, start another while loop iteration
                    level_flag = False
                    mainterm_parser.progress_through_levels_execute(mainterm)
                """
    
    def find_matching_mainterm_to_user_input(self, mainterm, user_input):
        """
        Args
            mainterm -> Mainterm object in focus.
            user_input -> User's choice on subterm to choose next.
        Returns:
            Subterm object if match to user's input is found.
        """
        # This function is called in progress_through_levels()
        # This function iterates through next level terms and returns the term with a 'title' or 'see' tag equal to user_input
        # This function returns a 'mainterm' object or raises LookupError
        # Find new mainterm object based on user choice above
        for subterm in mainterm["term"]:
            try:
                # Look for 'title' value that matches user input
                if subterm["title"] == user_input:
                    return subterm
            except KeyError:
                try:
                    # If no 'title' value, look for 'see' value that matches user input
                    if subterm["see"] == user_input:
                        return subterm
                except KeyError:
                    try:
                        if subterm["code"] == user_input:
                            return subterm
                    except KeyError:
                        # If no 'title' or 'see' match, raise error
                        
                        ### ADDED ###
                        global test_flag
                        #Indicate failed test
                        test_flag = False
                        
#                         raise LookupError("Could not find user's term in next_level_choices. Checked for a 'see', 'title', and 'code' tag. -> progress_through_levels()")

    
    def handle_bad_user_query(self, user_input, new_level_terms):
        """
        Args
            user_input -> User's choice on subterm to choose next.
            new_level_terms -> All possible choices for the user to make
        Returns:
            New user input that matches a usable term
        """
        # This function is called in progress_through_levels() and get_render_ask_next_level_terms()
        # If a user searches for a term that is not valid, give user three more trys
        # If user query term is not a valid term raise LookupError
        bad_query_flag = 0
        # User gets 5 query trys before throwing error
        while user_input not in new_level_terms and bad_query_flag <= 3:
            print("Your choice is not valid. Please try again.")
            user_input = input("Choose term : ")
            bad_query_flag += 1
            if bad_query_flag > 3:
                
                ### ADDED ###
                global test_flag
                #Indicate failed test
                test_flag = False
                
#                 raise LookupError("The term you've chosen does not match any valid options. Please try your query again.")
        return user_input
    
    def check_for_levels_in_mainterm(self, mainterm):
        """
        Args
            mainterm -> Mainterm object in focus.
        Returns
            - "level_1"
            - True
            - False
        """
        # This function checks for levels in the Mainterm object in focus.
        # It can return:
            # 'level_1' -> Indicates multiple terms on level 1 to be handled, no further levels.
            # True -> Indicates multiple levels to be handled.
            # False -> Indicates no levels to be handled.
        try:
            # Check for "term" key in mainterm. If no "term" key, return False (no levels)
            for subterm in mainterm["term"]:
                try:
                    # Check for "term" key in sub_term
                    if subterm["term"]:
                        # This indicates multiple levels
                        return True
                except KeyError:
                    pass
            try:
                # Check for level term on one of subterms
                if mainterm["term"][0]["_level"]:
                    # This indicates multiple level 1 terms
                    return "level_1"
            # If no mainterm["_level"] key, return False
            except KeyError:
                return False
        # If no mainterm["term"] key, return False
        except KeyError:
            return False
        
    def get_next_level_title_values(self, mainterm):
        """
        Args
            mainterm -> Mainterm object in focus.
        Returns
            List of "title" values found on next level of mainterm.
            If no next level, returns None.
        """
        # This function returns the "title" or "see" value of each next level subterm in a list
        # If the "mainterm" does not have subterms, the function returns None
        
        ### ADDED ###
        global test_flag
        
        next_level_terms = []
        try:
            # Get list of "title" values on the next level of subterms
            for level_term in mainterm["term"]:
                try:
                    # Get 'level' tag
                    title = level_term["title"]
                except KeyError:
                    try:
                        # If no 'level' tag, try to get 'see' tag
                        title = level_term["see"]
                    except KeyError:
                        try:
                            title = level_term["code"]
                        except KeyError:
                            # Raise error if cannot find text to use in 'title' or 'see'
                            
                            ### ADDED ###
                            #Indicate failed test
                            test_flag = False
                            
#                             raise LookupError("Could not determine which text to be used when retrieveing next level values. See get_next_level_title_values()")
                # Add term to terms list to return to user
                try:
                    next_level_terms.append(title)
                except:
                    
                    ### ADDED ###
                    test_flag = False
                    
        except KeyError:
            # If no next level terms, return None
            next_level_terms = None
        return next_level_terms
    
    def check_for_varying_subterm_structures(self, mainterm, set_like_object):
        """
        Args
            mainterm -> Mainterm object in focus.
            set_like_object -> set containing ordered dictionary keys which are being compared.
        Returns
            True
            ValueError
        """
        accounted_for_structures = [
            {'title', 'codes', '_level'}, # Accounted for in execute_single_level_5
            {'title', 'code', '_level'}, # Accounted for in execute_single_level_5
        ]
        # This function interates through each subterm to see if subterm structures vary from one another
        # It will raise ValueError if a mainterm structure is found that our code isn't prepared to handle
        # set_like_object is passed to the function and is a set of the first subterm's keys
        for term in mainterm["term"]:
            # Extracting keys of subterm
            term_keys = term.keys()
            # Iterating through subterm keys to check if structures vary
            for key in term_keys:
                # If the subterm's key is not in passed variable set_like_object
                # This means there is a subterm that contains a key that is different from the other subterm's keys
                if key not in set_like_object:
                    # Iterate through subterms and check that keys are not in accounted_for_structures
                    for subterm in mainterm["term"]:
                        # If not in accounted_for_structures, raise ValueError
                        if subterm.keys() in accounted_for_structures:
                            # This True value is used in execute_single_level()
                            return True
                        else:
                            
                            ### ADDED ###
                            global test_flag
                            #Indicate failed test
                            test_flag = False
                            
#                             raise ValueError("One subterm in the mainterm contains a key that is different from the other subterm keys and it is not accounted for.")
        # NOTE: Add functionality here to handle mainterm's with different structures
        # Could call an execute function here depending on structure found
    
    def get_render_ask_next_level_terms(self, mainterm, key_1, key_2=None):
        """
        Args
            mainterm -> Mainterm object in focus.
            key_1 -> String representing a dictionary key.
            key_2 -> String representing a dictionary key.
        Return
            User's choice of next level term, string.
        """
        # This function aggregates the next level's terms from a mainterm's "term" key based on
        # developer provided keys passed as key_1 and key_2
        # It then prints choices for the user and asks user to choose which subterm they want to use
        if key_2:
            text_choices = [term[key_1][key_2] for term in mainterm["term"]]
        else:
            try:
                text_choices = [term[key_1] for term in mainterm["term"]]
            except KeyError:
                try:
                    text_choices = [term["__text"] for term in mainterm["term"]]
                except KeyError:
                    try:
                        text_choices = [term["see"] for term in mainterm["term"]]
                    except KeyError:
                        # This mainterm has a blank subterm that does not contain a 'title' value
                        # This is used to catch that error
                        # Use this list to add terms that have errors like this
                        # May want to create a member variable to represent
                        if mainterm["title"] in ["Radiation Therapy"]:
                            text_choices = []
                            for subterm in mainterm["term"]:
                                try:
                                    text_choices.append(subterm["see"])
                                except KeyError:
                                    # Skipping one subterm that has no title value
                                    # Ex: Radiation Therapy'
                                    pass
                        else:
                            
                            ### ADDED ###
                            global test_flag
                            #Indicate failed test
                            test_flag = False
                            
#                             raise LookupError("Could not find term to use as 'title' of subterm. -> get_render_ask_next_level_terms()")
        print(text_choices)
        user_input = input("Choose term that most correlates to this medical case: ")
        if user_input in text_choices:
            return user_input
        else:
            return self.handle_bad_user_query(user_input, text_choices)

    def create_new_mainterm_generator(self, index):
        """
        Args
            index -> JSON document representing the PCS Index.
        Return
            Generator object of all 'mainterm' objects
        """
        # This function is called in: execute_single_level_1_1(), execute_group_2_1(), custom_search(), handle_pcs_table_component(), execute_group_2_1_1()
        # This function is used to create a new generator object of all 'mainterm' objects
        # It is needed in situations where we are re-querying the mainterms and have already iterated through a generator object
        # If we dont create a new generator object, the next iteration on that object will start where the last iteration left off
        letters_new = (letter_new for letter_new in index["ICD10PCS.index"]["letter"])
        return (mainterms_new for letter in letters_new for mainterms_new in letter["mainTerm"])
    
class Single_Level_Parser(Parser):
    def execute_single_level(self, mainterm):
        """
        Args
            mainterm -> Mainterm object in focus.
        Returns
            None.
        """
        ### ADDED ###
        global test_flag
        
        # This function points to a sub-function built to handle incoming mainterm structures
        # It returns ValueError if presented a mainterm structure it cannot handle
        # The first subterm is used to understand mainterm's structure
        # It is assumed that all subterms have the same key structure (which may not be true, Logan in progress of adding functionality)
        # Based on structure of subterms found, execute function that can handle structure
        first_subterm = mainterm["term"][0]
        if {"see", "_level"} == first_subterm.keys():
            self.check_for_varying_subterm_structures(mainterm, {"see", "_level"})
            self.execute_single_level_1(mainterm)
        elif {"title", "see", "_level"} == first_subterm.keys():
            self.check_for_varying_subterm_structures(mainterm, {"title", "see", "_level"})
            self.execute_single_level_2(mainterm)    
        elif {"use", "_level"} == first_subterm.keys():
            self.check_for_varying_subterm_structures(mainterm, {"use", "_level"})
            self.execute_single_level_3(mainterm)
        elif {"code", "_level"} == first_subterm.keys():
            self.check_for_varying_subterm_structures(mainterm, {"code", "_level"})
            self.execute_single_level_4(mainterm)
        elif {"title", "code", "_level"} == first_subterm.keys():
            self.check_for_varying_subterm_structures(mainterm, {'title', 'code', '_level'})
            self.execute_tree(mainterm)
        elif {"title", "codes", "_level"} == first_subterm.keys():
            varying_structures = self.check_for_varying_subterm_structures(mainterm, {"title', 'codes', '_level"})
            if varying_structures:
                self.execute_single_level_5(mainterm)
            else:
                ### ADDED ###
                #Indicate failed test
                test_flag = False
                
#                 raise ValueError("Code not prepared for mainterm's keys -> execute_single_level()")
        else:
            ### ADDED ###
            #Indicate failed test
            test_flag = False
            
#             raise ValueError("Code not prepared mainterm's keys -> execute_single_level()") 

            
    def execute_single_level_1(self, mainterm):
        print("--------execute_single_level_1--------")
        """
        Args
            mainterm -> Mainterm object in focus.
        Returns
            None.
        """
        # This function is called in execute_single_level()
        # This function handles a "see" tag with or without children
        # It points to a sub-function that is built to handle the structure
        # Assumes subterms all have same key structure (determined by check_for_varying_subterm_structures) (Logan working on)
        first_subterm = mainterm["term"][0]
        # If type str this indicates the "see" key has no children, execute function built to handle.
        if isinstance(first_subterm["see"], str):
            self.execute_single_level_1_1(mainterm) 
        # If "see" key has children, execute based on structure returned
        elif {"tab", "__text"} == first_subterm["see"].keys():
            self.execute_single_level_1_2(mainterm)
        elif {"codes", "__text"} == first_subterm["see"].keys():
            self.execute_single_level_1_2(mainterm)
        else:
            
            ### ADDED ###
            global test_flag
            #Indicate failed test
            test_flag = False
            
#             raise ValueError("Code not prepared for mainterm key structure -> execute_single_level_1")

    def execute_single_level_1_1(self, mainterm):
        print("--------execute_single_level_1_1--------")
        """
        Args
            mainterm -> Mainterm object in focus.
        Returns
            None.
        """
        # This function is called in execute_single_level_1()
        # This function handles mainterm objects containing a "see" key that does not have children
        # Get user's selection of new mainterm of focus
        
        ### Added ###
        # Taking this function out because it is asking the user to choose a level term
        # We are not testing for levels in this script
        global test_flag
        test_flag = "No Test"
        
        """
        # Ensure term_found_flag is False
        self.term_found_flag = False
        user_input = self.get_render_ask_next_level_terms(mainterm, "see")
        # Re-query the 'mainterms' generator to find the mainterm that matches user's selection
        # Create new generator object so that new search starts at the beginning of the mainterms
        mainterms = self.create_new_mainterm_generator(index)
        for new_mainterm in mainterms:
            if new_mainterm["title"] == user_input:
                self.term_found_flag = True
                self.execute_tree(new_mainterm)
                break
        # Trying to execute "see" tag with specific structure (Ex: "Brachytherapy, CivaSheet(R)")
        if not self.term_found_flag:
            try:
                for new_mainterm_1 in mainterm["term"]:
                    if new_mainterm_1["see"] == user_input:
                        self.term_found_flag = True
                        mainterm_parser.execute_group_2_1(new_mainterm_1)
            except KeyError:
                raise LookupError("Code not prepared for mainterm key structure -> execute_single_level_1_1()")
        # If we didnt find a valid response, raise LookupError
        if not self.term_found_flag:
            raise LookupError("Code not prepared for mainterm key structure -> execute_single_level_1_1")
        """
    
    def execute_single_level_1_2(self, mainterm):
        print("--------execute_single_level_1_2--------")
        """
        Args
            mainterm -> Mainterm object in focus.
        Returns
            None.
        """
        # This function is called in execute_single_level_1()
        # This function handles mainterm objects containing a "see" key that does have children
        # See keys require a second query for a new lookup term
        
        ### Added ###
        # Taking this function our because it is asking the user to choose a level term
        # We are not testing for levels in this script
        global test_flag
        test_flag = "No Test"
        
        """
        # Ensure term_found_flag is False
        self.term_found_flag = False
        # Get user's selection of new mainterm of focus
        user_input = self.get_render_ask_next_level_terms(mainterm, "see", "__text")
        # Searching subterms to find new Mainterm object in focus
        for new_mainterm in mainterm["term"]:
            if new_mainterm["see"]["__text"] == user_input:
                # Executing with new mainterm
                self.term_found_flag = True
                self.execute_tree(new_mainterm)
                break
        # If we didnt find a valid response, raise ValueError
        if not self.term_found_flag:
            raise LookupError("Code not prepared for mainterm key structure -> execute_single_level_1_2")
        """     
    def execute_single_level_2(self, mainterm):
        print("--------execute_single_level_2--------")
        """
        Args
            mainterm -> Mainterm object in focus.
        Returns
            None
        """
        # This function is called in execute_single_level()
        # This function handles a "see" tag that contains levels
        
        ### Added ###
        # Taking this function our because it is asking the user to choose a level term
        # We are not testing for levels in this script
        global test_flag
        test_flag = "No Test"
        
        """
        # Ensure term_found_flag is False
        self.term_found_flag = False
        # Get user's selection of new mainterm of focus
        user_input = self.get_render_ask_next_level_terms(mainterm, "title")
        # Searching subterms to find new Mainterm object in focus
        for new_mainterm in mainterm["term"]:
            if new_mainterm["title"] == user_input:
                # Executing with new mainterm
                self.term_found_flag = True
                self.execute_tree(new_mainterm)
                break
        # If we didnt find a valid response, raise ValueError
        if not self.term_found_flag:
            raise LookupError("Code not prepared for mainterm key structure -> execute_single_level_2")
        """
                
    def execute_single_level_3(self, mainterm):
        print("--------execute_single_level_3--------")
        """
        Args
            mainterm -> Mainterm object in focus.
        Returns
            None
        """
        # This function is called in execute_single_level()
        # This function handles a "use" tag with or without children
        
        
        ### Added ###
        # Taking this function our because it is asking the user to choose a level term
        # We are not testing for levels in this script
        global test_flag
        test_flag = "No Test"
        
        """
        # Ensure term_found_flag is False
        self.term_found_flag = False
        # Assumes subterms all have same key structure (determined by check_for_varying_subterm_structures)
        first_subterm = mainterm["term"][0]
        # If type str, this indicates the "use" key has no children
        if isinstance(first_subterm["use"], str):
            # Returns string of new mainterm of focus based on user response in get_render_ask_next_level_terms()
            user_input = self.get_render_ask_next_level_terms(mainterm, "use")
            # Search subterms to find new Mainterm object in focus
            for new_mainterm in mainterm["term"]:
                # Execute with new mainterm
                if new_mainterm["use"] == user_input:
                    self.term_found_flag = True
                    self.execute_tree(new_mainterm)
                    break
            # If we didnt find a valid response, raise ValueError
            if not self.term_found_flag:
                raise LookupError("Code not prepared for mainterm key structure -> execute_single_level_3")
        else:
            # If "use" key has children, execute based on structure returned
            if {"tab", "__text"} == first_subterm["use"].keys():
                # Returns string of new mainterm of focus based on user response in get_render_ask_next_level_terms()
                user_input = self.get_render_ask_next_level_terms(mainterm, "use", "__text")
                for new_mainterm in mainterm["term"]:
                    if new_mainterm["use"]["__text"] == user_input:
                        self.term_found_flag = True
                        self.execute_tree(new_mainterm)
                        break 
                # If we didnt find a valid response, raise ValueError
                if not self.term_found_flag:
                    raise LookupError("Code not prepared for mainterm key structure -> execute_single_level_3")
            else:
                raise ValueError("Code not prepared for mainterm key structure -> execute_single_level_3")
        """
        
    def execute_single_level_4(self, mainterm):
        print("--------execute_single_level_4--------")
        """
        Args
            mainterm -> Mainterm object in focus.
        Returns
            Information to user in form of print(). Will integrate with PCS Table.
        """
        # Execute function
        # This function is called in execute_single_level()
        # This function handles mainterm objects with subterms that contain a "code" key
        # Get number of final code's available 
#         len_choices = len(mainterm["term"])
#         # Get list of available codes
#         choices = [term["code"] for term in mainterm["term"]]
#         # Return codes to user
#         print(f"""
# You have {len_choices} choices for final codes related to term '{mainterm['title']}'. Choose the code that works best for this medical case. No further guidance given.""")
#         print(choices)
#         # Check to see if given a 'qualifier' or 'device'
#         # Always found in 'see' key
#         if self.pcs_component:
#             print(f"Use {self.pcs_component[0]} '{self.pcs_component[1]}' in the PCS table.")
#             # Resetting pcs_component flag
#             self.pcs_component = False

        ### ADDED ###
        global test_flag
        # Testing to see if code can execute what it is intended to execute
        try:
            len_choices = len(mainterm["term"])
            choices = [term["code"] for term in mainterm["term"]]
            if self.pcs_component:
                # Testing components are present
                comp_1 = self.pcs_component[0]
                comp_2 = self.pcs_component[1]
                # Resetting pcs_component flag
                self.pcs_component = False
            test_flag = True
        except:
            # Indicates failed test
            test_flag = False

    def execute_single_level_5(self, mainterm):
        print("--------execute_single_level_5--------")
        """
        Args
            mainterm -> Mainterm object in focus.
        Returns
            Information to user in form of print(). Will integrate with PCS Table.
        """
        # Execute function
        # This function is called in execute_single_level()
        # This function is used to handle multiple subterm structures in the same mainterm
        # This function only works when 'code' and 'codes' are available as subterms in a mainterm's 'term' list
        # Add functionality here to handle other varying structures as need
        
        ### ADDED ###
        global test_flag

        # Check if only 'code' and 'codes' are available in subterms
        # We are also checking for if 'see' is available
        # Depending on outcome, execute function built to handle
        # Used as flag for code to determine structure
        see_structure_flag = False
        for subterm in mainterm["term"]:
            try:
                # 'code' key is available
                if subterm["code"]:
                    # Dont change see_structure_flag
                    pass
            except KeyError:
                try:
                    # 'codes' key is available
                    if subterm["codes"]:
                        # Dont change see_structure_flag
                        pass
                except KeyError:
                    try:
                        # 'see' key is available
                        if subterm["see"]:
                            # Change see structure flag
                            see_structure_flag = True
                    except:
                        pprint(mainterm)
                        raise LookupError("Unknown subterm structure found. Need function to handle -> execute_single_level_5()")
        # Ex: 'Abortion'
        if not see_structure_flag:
            self.execute_single_level_5_1(mainterm)
        # Ex: 'Phototherapy'
        else:
            # Execute function that can handle 'see' and 'codes' structures on the same level
#             user_input = self.get_render_ask_next_level_terms(mainterm, "title")
#             for mainterm_1 in mainterm["term"]:
#                 if mainterm_1["title"] == user_input:
#                     self.execute_tree(mainterm_1, single_level_check=True)
#                     break
            
            ### ADDED ###
            try:
                titles = [i for i["title"] in mainterm["term"]]
                test_flag = True
            except:
                test_flag = False
                    
    def execute_single_level_5_1(self, mainterm):
        print("--------execute_single_level_5_1--------")
        """
        Args
            mainterm -> Mainterm object in focus.
        Returns
            Information to user in form of print(). Will integrate with PCS Table.
        """
        # Execute function
        # This function handles mainterms with 'code' and 'codes' keys on the same level of subterms
        # Ex: 'Abortion'
#         final_codes = []
#         partial_codes = []
#         for term in mainterm["term"]:
#             try:
#                 # Create list of final codes
#                 code_data = (term["code"], term["title"])
#                 final_codes.append(code_data)
#             except KeyError:
#                 # Create list of partial codes
#                 codes_data = (term["codes"], term["title"])
#                 partial_codes.append(codes_data)
#         # Return codes to user
#         if len(final_codes) > 0:
#             print(f"""
# Choices for Final codes: {final_codes}""")
#         if len(partial_codes) > 0:
#             print(f"Choices for un-finished codes: {partial_codes}")
#         # If nothing is found, raise LookupError
#         if len(partial_codes) == 0 and len(final_codes) == 0:
#             raise LookupError("Not prepared for mainterm structure -> execute_single_level_5()")
#         # Check to see if given a 'qualifier' or 'device'
#         # Always found in 'see' key
#         if self.pcs_component:
#             print(f"Use {self.pcs_component[0]} '{self.pcs_component[1]}' in the PCS table.")
#             # Resetting pcs_component flag
#             self.pcs_component = False

        ### ADDED ###
        global test_flag
        try:
            final_codes = []
            partial_codes = []
            for term in mainterm["term"]:
                try:
                    # Create list of final codes
                    code_data = (term["code"], term["title"])
                    final_codes.append(code_data)
                except KeyError:
                    # Create list of partial codes
                    codes_data = (term["codes"], term["title"])
                    partial_codes.append(codes_data)
            if self.pcs_component:
                # Testing components are present
                comp_1 = self.pcs_component[0]
                comp_2 = self.pcs_component[1]
                # Resetting pcs_component flag
                self.pcs_component = False
            test_flag = True
        except:
            test_flag = False

class Mainterm_Parser(Parser):
    def parent_execute(self, mainterm):
        """
        Args
            mainterm -> Mainterm object in focus.
        Returns
            None
        """
        # This function is called when there are no levels in the Mainterm object in focus
        # Depending on the mainterm's key structure, an execute function built to handle given structure is called
        if {"title", "use"} == mainterm.keys():
            self.execute_group_1(mainterm)
        elif {"title", "see"} == mainterm.keys():
            self.execute_group_2(mainterm)
        elif {"title", "term"} == mainterm.keys():
            self.execute_tree(mainterm)
        elif {"title", "tab"} == mainterm.keys():
            self.execute_group_5(mainterm)
        elif {"title", "codes"} == mainterm.keys():
            self.execute_group_6(mainterm)
        elif {"title", "code"} == mainterm.keys():
            self.execute_group_4(mainterm)
        elif {"title", "code", "term"} == mainterm.keys():
            self.execute_group_7(mainterm)
        elif {"title", "code", "_level"} == mainterm.keys():
            self.execute_group_4(mainterm)
        elif {"see", "_level"} == mainterm.keys():
            self.execute_group_2(mainterm)
        elif {"use", "_level"} == mainterm.keys():
            self.execute_group_1(mainterm)
        elif {"title", "see", "_level"} == mainterm.keys():
            self.execute_group_2(mainterm)
        else:
            
            ### ADDED ###
            global test_flag
            #Indicate failed test
            test_flag = False
            
#             pprint(mainterm)
#             raise LookupError("Code not prepared for mainterm key structure -> parent_execute()")
            
    def progress_through_levels_execute(self, mainterm):
        print("--------progress_through_levels_execute--------")
        # This function is called in Parser().progress_through_levels()
        # It uses sub-functions defined in the Mainterm_Parser() class to complete queries
        # Depending on the key structure of the passed 'mainterm', this function will call sub-function built to handle
        
        ### ADDED ###
        global test_flag
        
        if {'title', 'codes', '_level'} == mainterm.keys():
            self.execute_group_6(mainterm)  
        elif {'title', 'code', '_level'} == mainterm.keys():
            self.execute_group_4(mainterm)
        elif {'see', '_level'} == mainterm.keys():
            self.execute_group_2(mainterm)
        elif {'use', '_level'} == mainterm.keys():
            self.execute_group_1(mainterm)
        elif {'title', 'see', '_level'} == mainterm.keys():
            self.execute_group_2(mainterm)
        elif {'code', '_level'} == mainterm.keys():
            self.execute_group_4(mainterm)
        elif {'title', 'tab', '_level'} == mainterm.keys():
            self.execute_group_5(mainterm)
#         elif {"title", "term", "level"}:
#             pass
        elif {"_level"} == mainterm.keys():
        
            ### ADDED ###
            #Indicate failed test
            test_flag = False
            
#             print(mainterm)
#             raise LookupError("Code not prepared for mainterm key structure -> progress_through_levels_execute()")
        else:
        
            ### ADDED ###
            #Indicate failed test
            test_flag = False
            
#             pprint(mainterm)
#             raise LookupError("Code not prepared for mainterm key structure -> progress_through_levels_execute()")
    
    def execute_group_1(self, mainterm):
        print("--------execute_group_1--------")
        """
        Args
            mainterm -> Mainterm object in focus.
        Returns
            None.
        """
        # This function is called in parent_execute() and progress_through_levels_execute()
        # This function points to sub-functions that handle mainterm objects containing a "use" key
        # If "use" key returns a string then it has no children, execute function built to handle.
        if isinstance(mainterm["use"], str):
            self.execute_group_1_1(mainterm)
        # If the "use" key has children, execute based on children's key structure
        elif {"tab", "__text"} == mainterm["use"].keys():
            self.execute_group_1_2(mainterm)
        else:
            
            ### ADDED ###
            global test_flag
            #Indicate failed test
            test_flag = False
            
#             pprint(mainterm)
#             raise LookupError("Code not prepared for mainterm key structure -> execute_group_1()")

    
    def execute_group_1_1(self, mainterm):
        print("--------execute_group_1_1--------")
        """
        Args
            mainterm -> Mainterm object in focus.
        Returns
            Information to user in form of print(). Will integrate with PCS Table.
        """
        # Execute Function
        # This function is called in execute_group_1()
        # This function is used to handle mainterm objects that contain a "use" key that has no children
        # This function returns a term that corresponds to a code in a PCS Table
        # This term may be a:
            # Body Part Key
            # Device Key
            # Substance Key
            # Device Aggregation Table
                # Contains entries that correlate a specific PCS device value to a general device value
                # which can be used in PCS tables containing only general values
#         print(f"""
# Use the code associated with term '{mainterm['use']}' in the PCS Table.""")
#         # Check to see if given a 'qualifier' or 'device'
#         # Always found in 'see' key
#         if self.pcs_component:
#             print(f"Use {self.pcs_component[0]} '{self.pcs_component[1]}' in the PCS table.")
#             # Resetting pcs_component flag
#             self.pcs_component = False

        ### ADDED ###
        global test_flag
        try:
            test = mainterm['use']
            if self.pcs_component:
                comp_1 = self.pcs_component[0]
                comp_2 = self.pcs_component[1]
                # Resetting pcs_component flag
                self.pcs_component = False
                # Indicate successful test
            test_flag = True
        except KeyError:
            # Indicate failed test
            test_flag = False

    def execute_group_1_2(self, mainterm):
        print("--------execute_group_1_2--------")
        """
        Args
            mainterm -> Mainterm object in focus.
        Returns
            Information to user in form of print(). Will integrate with PCS Table.
        """
        # Execute Function
        # This function is called in execute_group_1()
        # This function is used to handle mainterm objects that contain a "use" key that has no children
        # This function returns the PCS Table which the user needs to use
        # It also returns text that may correspond to a pos. 4-7 code in a PCS Row inside that PCS Table
#         table = mainterm["use"]["tab"]
#         text = mainterm["use"]["__text"]
#         print(f"""
# Go to table {table}, use PCS Row containing text '{text}' in pos. 4-7 values.""")
#         # Check to see if given a 'qualifier' or 'device'
#         # Always found in 'see' key
#         if self.pcs_component:
#             print(f"Use {self.pcs_component[0]} '{self.pcs_component[1]}' in the PCS table.")
#             # Resetting pcs_component flag
#             self.pcs_component = False

        ### ADDED ###
        global test_flag
        try:
            table = mainterm["use"]["tab"]
            text = mainterm["use"]["__text"]
            if self.pcs_component:
                print(f"Use {self.pcs_component[0]} '{self.pcs_component[1]}' in the PCS table.")
                # Resetting pcs_component flag
                self.pcs_component = False
                # Indicate successful test
                test_flag = True
        except:
            # Indicate failed test
            test_flag = False
        
    
    def execute_group_2(self, mainterm):
        print("--------execute_group_2--------")
        """
        Args
            mainterm -> Mainterm object in focus.
        Returns
            None.
        """
        # This function is called in parent_execute() and progress_through_levels_execute()
        # This function points to sub-functions that handle mainterm objects containing a "see" key
        # If mainterm's "see" key returns a string then it has no children, execute function built to handle.
        if isinstance(mainterm["see"], str):
            self.execute_group_2_1(mainterm)
        # If mainterm's "see" key has children, execute based on structure returned
        elif {"codes", "__text"} == mainterm["see"].keys():
            self.execute_group_2_2(mainterm)
        elif {"tab", "__text"} == mainterm["see"].keys():
            self.execute_group_2_3(mainterm)
        else:
            
            ### ADDED ###
            global test_flag
            #Indicate failed test
            test_flag = False
            
#             pprint(mainterm)
#             raise LookupError("Code not prepared for mainterm key structure -> progress_through_levels_execute()")

    def execute_group_2_1(self, mainterm):
        print("--------execute_group_2_1--------")
        """
        Args
            mainterm -> Mainterm object in focus.
        Returns
            None.
        """
        
        ### ADDED ###
        global test_flag
        
        # This function is called in execute_group_2()
        # This function handles mainterm objects with a "see" key
        new_term = mainterm["see"]
        print(f"--Redirected to term '{new_term}'.") # Logging
        # Create new generator object
        mainterms = self.create_new_mainterm_generator(index)
        # Ensure term_found_flag is False
        self.term_found_flag = False
        # Re-query mainterms to find new_mainterm object with matching "title" 
        # "title" value must match the original mainterm's "see" value
        for new_mainterm_1 in mainterms:
            # If mainterm object's 'title' value is equal to our new search term, execute
            # Using capitalized case search term for specific use cases (Ex: "Diagnostic Audiology","Diagnostic imaging","Diagnostic radiology","Radiology, diagnostic","Nuclear scintigraphy")
            if new_mainterm_1["title"] == new_term or new_mainterm_1["title"] == new_term.capitalize():
                print(f"--Found '{new_term}' on first attempt") # Logging
                self.term_found_flag = True
                self.execute_tree(new_mainterm_1, single_level_check=True)
                break   
        # If we did not find a match
        # See if structure of new_term is two separate terms separated by commas
        if not self.term_found_flag:
            # Split value returned from "see" key to re-query data for match
            # When a "see" value has multiple terms separated by ", " that means they have given us a subterm(s) (second, third term) to look for within the mainterm (first term)
            split_term = new_term.split(", ")
            if len(split_term) == 2:
                # If there are two terms, execute function that can handle
                self.term_found_flag = True
                self.execute_group_2_1_1(split_term)
            elif len(split_term) == 3:
                # If there are three terms, execute function that can handle
                self.term_found_flag = True
                self.custom_search(new_term)
        # If we have still not found a match 
        if not self.term_found_flag:
            # Isolate first word from new_term
            first_word = new_term.split(" ")[0]
            # If first word is introduction... (Ex: 'Drotrecogin alfa, infusion', 'Eptifibatide, infusion')
            if first_word == "Introduction":
                """
                self.term_found_flag = True
                print("NEED FUNCTION TO HANDLE. INTEGRATE TO PCS TABLE") 
                """
                # Using to avoid raising error later. This line is only for the debugging script.
                self.term_found_flag = True 
                
                ### ADDED ###
                # Need to add function here that handles mainterm's with 'title' value equal to "Introduction..."
                test_flag = False
                
        # If we still havent found a match
        if not self.term_found_flag:
            # Look for words signifying a certain structure (Ex: Core needle biopsy, Punch biopsy)
            if "with qualifier" in new_term or "with device" in new_term:
                self.term_found_flag = True
                self.handle_pcs_table_component(new_term)
        # If still now match, throw LookupError
        if not self.term_found_flag:
            
            ### ADDED ###
            #Indicate failed test
            test_flag = False
            
#             raise LookupError("Code not prepared for mainterm key structure -> execute_group_2_1()")

    def custom_search(self, new_term):
        print("--------custom_search--------")
        """
        Args
            new_term -> "see" value that contains 3 separate search terms in one string.
        Returns
            None.
        """
        # This function is called in execute_group_2_1()
        # These are custom searches that are hard coded because of there unique "see" value structure 
        # 'Ligation, hemorrhoid'
        
        ### ADDED ###
        global test_flag
        
        if new_term == "Occlusion, Lower Veins, Hemorrhoidal Plexus":
            """
            # Create new 'mainterms' generator object
            mainterms = self.create_new_mainterm_generator(index)
            # Find first search terms object with 'title' value of "Occlusion"
            for new_mainterm_2 in mainterms:
                if new_mainterm_2["title"] == "Occlusion":
                    # Find second search terms object with 'title' value of "Vein"
                    for new_mainterm_3 in new_mainterm_2["term"]:
                        if new_mainterm_3["title"] == "Vein":
                            # Find third search term's object with 'title' value of "Lower"
                            for new_mainterm_4 in new_mainterm_3["term"]:
                                if new_mainterm_4["title"] == "Lower":
                                    code = new_mainterm_4["codes"]
                                    # Used in execute function to notify that Index gave us a PCS Table component
                                    self.pcs_component = ("qualifier", "Hemorrhoidal Plexus")
                                    self.execute_group_6(new_mainterm_4)
            """
            ### ADDED ###
            test_flag = True
            
        # 'Myocardial Bridge Release'
        elif new_term == "Release, Artery, Coronary":
            """
            # Create new 'mainterms' generator object
            mainterms = self.create_new_mainterm_generator(index)
            for new_mainterm_2 in mainterms:
                # Search for 'mainterm' object with 'title' value of "Release"
                if new_mainterm_2["title"] == "Release":
                    for new_mainterm_3 in new_mainterm_2["term"]:
                        # Search for subterm object with 'title' value of "Artery"
                        if new_mainterm_3["title"] == "Artery":
                            for new_mainterm_4 in new_mainterm_3["term"]:
                                # Search for subterm object with 'title' value of "Coronary"
                                if new_mainterm_4["title"] == "Coronary":
                                    self.execute_tree(new_mainterm_4)
            """
            ### ADDED ###
            test_flag = True
        
        # Psychotherapy -> Individual -> Mental Health Services
        elif new_term == "Psychotherapy, Individual, Mental Health Services":
            """
            # Create new 'mainterms' generator object
            mainterms = self.create_new_mainterm_generator(index)
            for new_mainterm_2 in mainterms:
                # Search for 'mainterm' object with 'title' value of "Release"
                if new_mainterm_2["title"] == "Psychotherapy":
                    for new_mainterm_3 in new_mainterm_2["term"]:
                        # Search for subterm object with 'title' value of "Artery"
                        if new_mainterm_3["title"] == "Individual":
                            for new_mainterm_4 in new_mainterm_3["term"]:
                                # Search for subterm object with 'title' value of "Coronary"
                                try:
                                    if new_mainterm_4["title"] == "Mental Health Services":
                                        self.execute_tree(new_mainterm_4, single_level_check=True)
                                except KeyError:
                                    if new_mainterm_4["see"] == "Mental Health Services":
                                        self.execute_tree(new_mainterm_4, single_level_check=True)
            """
            ### ADDED ###
            test_flag = True
            
        else:
            ### ADDED ###
            #Indicate failed test
            test_flag = False
            
#             raise LookupError("Do not have a function build to handle lookup term '{new_term}' -> custom_search()")

    def handle_pcs_table_component(self, new_term):
        print("--------handle_pcs_table_component--------")
        """
        Args
            new_term -> "see" value that contains a string representing a new search term and PCS Table component.
        Returns
            None.
        """
        # This function is called in execute_group_2_1()
        # This function parses a string containing information returned from a "see" tag
        # The string tells us what the new search term is, the PCS column header to be used (pos), and the PCS component to be used
        # Ex: 'Core needle biopsy', 'Punch biopsy'
        # Spliting text returned as value of "see" tag
        split_text = new_term.split(" ")
        # Finding new search term (always first word)
        new_search_term = split_text[0]
        # Finding pcs table column header to be used
        pcs_pos = split_text[2]
        # Finding pcs term to be used under table's column header
        pcs_component = " ".join(split_text[3:])
        # Create new generator object
        mainterms = self.create_new_mainterm_generator(index)
        # Finding search term and executing
        for mainterm_1 in mainterms:
            if mainterm_1["title"] == new_search_term:
                # Flag to let output know if they gave us a pcs component
                # tuple will always have the 1st position be the pos word and second position be the component
                self.pcs_component = (pcs_pos, pcs_component)
                self.execute_tree(mainterm_1, single_level_check=True)

    def execute_group_2_1_1(self, split_term):
        print("--------execute_group_2_1_1--------")
        """
        Args
            split_term -> two individual search terms gathered from a "see" value
        Returns
            None.
        """
        # This function is called in execute_group_2_1()
        # This function re-queries the data to find new_mainterm given by "see" value
        # Once it finds the parent_search_term, it will find the child_search_term within the parent_search_term
        # Ensure term_found_flag is False
        self.term_found_flag = False
        # Unpack split_term
        parent_search_term, child_search_term = split_term
        print(f"--Looking for new parent term: {parent_search_term}") # Logging
        # Re-query the database to find first search term
        # Create new generator object
        mainterms = self.create_new_mainterm_generator(index)
        for new_mainterm_2 in mainterms:
            if new_mainterm_2["title"] == parent_search_term:
                # If find first search term, look for second search term
                for subterm in new_mainterm_2["term"]:
                    if subterm["title"] == child_search_term:
                        # If find second search term, execute
                        print(f"--Found child term: {child_search_term}")
                        self.term_found_flag = True
                        self.execute_tree(subterm)
                        break
        if not self.term_found_flag:
            ### ADDED ###
            global test_flag
            #Indicate failed test
            test_flag = False
            
#             raise LookupError("Code not prepared for mainterm key structure -> execute_group_2_1_1()")
    
    def execute_group_2_2(self, mainterm):
        print("--------execute_group_2_2--------")
        """
        Args
            mainterm -> Mainterm object in focus.
        Returns
            Information to user in form of print(). Will integrate with PCS Table.
        """
        # Execute Function
        # This function is called in execute_group_2()
        # This function handles mainterm objects with a "see" key that contains "codes" and "__text" keys
#         code = mainterm["see"]["codes"]
#         text = mainterm["see"]["__text"]
#         print(f"""
# Go to table '{code[:3]}', located at '{text}'.
# Position 4 and greater code(s): '{code[3:]}'
# Each code corresponds to it's respective position number.""")
#         # Check to see if given a 'qualifier' or 'device'
#         # Always found in 'see' key
#         if self.pcs_component:
#             print(f"Use {self.pcs_component[0]} '{self.pcs_component[1]}' in the PCS table.")
#             # Resetting pcs_component flag
#             self.pcs_component = False
        
        
        ### ADDED ###
        global test_flag
        try:
            code = mainterm["see"]["codes"]
            text = mainterm["see"]["__text"]
            if self.pcs_component:
                comp_1 = self.pcs_component[0]
                comp_2 = self.pcs_component[1]
                # Resetting pcs_component flag
                self.pcs_component = False
            test_flag = True
        except:
            test_flag = False
        
    def execute_group_2_3(self, mainterm):
        print("--------execute_group_2_3--------")
        """
        Args
            mainterm -> Mainterm object in focus.
        Returns
            Information to user in form of print(). Will integrate with PCS Table.
        """
        # Execute Function
        # This function is called in execute_group_2()
        # This function handles mainterm objects with a "see" key that contains "tab" and "__text" keys
#         table = mainterm["see"]["tab"]
#         text = mainterm["see"]["__text"]
#         print(f"""
# Go to table '{table}', located at sections '{text}'.
# The 'sections' may also correspond to a pos. 4-7 value in a PCS Row in the given PCS Table""")
#         # Check to see if given a 'qualifier' or 'device'
#         # Always found in 'see' key
#         if self.pcs_component:
#             print(f"Use {self.pcs_component[0]} '{self.pcs_component[1]}' in the PCS table.")
#             # Resetting pcs_component flag
#             self.pcs_component = False
    
        ### ADDED ###
        global test_flag
        try:
            table = mainterm["see"]["tab"]
            text = mainterm["see"]["__text"]
            if self.pcs_component:
                comp_1 = self.pcs_component[0]
                comp_2 = self.pcs_component[1]
                # Resetting pcs_component flag
                self.pcs_component = False
            test_flag = True
        except:
            test_flag = False
            
    def execute_group_4(self, mainterm):
        print("--------execute_group_4--------")
        """
        Args
            mainterm -> Mainterm object in focus.
        Returns
            Information to user in form of print(). Will integrate with PCS Table.
        """
        # Execute Function
        # This function is called in Mainterm_Parser().parent_execute() and Mainterm_Parser().progress_through_levels_execute()
        # This function handles a mainterm object with keys:
            # {"title", "code"}
            # {'code', '_level'}
            # {'title', 'code', '_level'}
#         try:
#             # If "mainterm" has a "title" key, display "title" in return
#             print(f"""
# Use final code: {mainterm['code']} with description '{mainterm['title']}'.""")
#         except KeyError:
#             # If no title key, only display "code"
#             print(f"""
# Use final code: {mainterm['code']}. No further information given.""")
#         # Check to see if given a 'qualifier' or 'device'
#         # Always found in 'see' key
#         if self.pcs_component:
#             print(f"Use {self.pcs_component[0]} '{self.pcs_component[1]}' in the PCS table.")
#             # Resetting pcs_component flag
#             self.pcs_component = False

        ### ADDED ###
        global test_flag
        try:
            try:
                code = mainterm['code']
                title = mainterm['title']
            except KeyError:
                code = mainterm['code']
            if self.pcs_component:
                comp_1 = self.pcs_component[0]
                comp_2 = self.pcs_component[1]
                # Resetting pcs_component flag
                self.pcs_component = False
            test_flag = True
        except:
            test_flag = False
        
    def execute_group_5(self, mainterm):
        print("--------execute_group_5--------")
        """
        Args
            mainterm -> Mainterm object in focus.
        Returns
            Information to user in form of print(). Will integrate with PCS Table.
        """
        # Execute Function
        # This function is called in Mainterm_Parser().parent_execute() and Mainterm_Parser().progress_through_levels_execute()
        # This function handles a mainterm object with keys:
            # {"title", "tab"}
            # {'title', 'tab', '_level'}
#         table = mainterm["tab"]
#         print(f"""
# Go to table: {table}. No additional guidance given.""")
#         # Check to see if given a 'qualifier' or 'device'
#         # Always found in 'see' key
#         if self.pcs_component:
#             print(f"Use {self.pcs_component[0]} '{self.pcs_component[1]}' in the PCS table.")
#             # Resetting pcs_component flag
#             self.pcs_component = False

        ### ADDED ###
        global test_flag
        try:
            table = mainterm["tab"]
            if self.pcs_component:
                comp_1 = self.pcs_component[0]
                comp_2 = self.pcs_component[1]
                # Resetting pcs_component flag
                self.pcs_component = False
            test_flag = True
        except:
            test_flag = False

    def execute_group_6(self, mainterm):
        print("--------execute_group_6--------")
        """
        Args
            mainterm -> Mainterm object in focus.
        Returns
            Information to user in form of print(). Will integrate with PCS Table.
        """
        # Execute Function
        # This function is called in Mainterm_Parser().parent_execute() and Mainterm_Parser().progress_through_levels_execute()
        # This function handles a mainterm object with keys:
            # "title", "codes"}
            # {'title', 'codes', '_level'}
#         code = mainterm["codes"]
#         print(f"""
# Go to table: {code[:3]}
# Position 4 and greater code(s): '{code[3:]}'
# Each code corresponds to it's respective position number.""")
#         # Check to see if given a 'qualifier' or 'device'
#         # Always found in 'see' key
#         if self.pcs_component:
#             print(f"Use {self.pcs_component[0]} '{self.pcs_component[1]}' in the PCS table.")
#             # Resetting pcs_component flag
#             self.pcs_component = False

        ### ADDED ###
        global test_flag
        try:
            code = mainterm["codes"]
            if self.pcs_component:
                comp_1 = self.pcs_component[0]
                comp_2 = self.pcs_component[1]
                # Resetting pcs_component flag
                self.pcs_component = False
            test_flag = True
        except:
            test_flag = False
        

    def execute_group_7(self, mainterm):
        print("--------execute_group_7--------")
        """
        Args
            mainterm -> Mainterm object in focus.
        Returns
            Information to user in form of print(). Will integrate with PCS Table.
        """
        # Execute Function
        # This function is called in Mainterm_Parser().parent_execute()
        # This function handles a "mainterm" with keys:
            # {"title", "code", "term"}
        # Get "code", "title" values for next level of terms
#         code = mainterm["code"]
#         level_1_terms = self.get_next_level_title_values(mainterm)
#         # Display "code" to user and ask if any next_level_terms apply to this medical case
#         user_input = input(f"""
# Use code: {code}. If this case involves any of the following terms, {level_1_terms}, press 'y' now. If not, press 'n' to exit.""")
#         if user_input == "y":
#             # If next level terms apply, execute
#             self.execute_tree(mainterm)
#         # Check to see if given a 'qualifier' or 'device'
#         # Always found in 'see' key
#         if self.pcs_component:
#             print(f"Use {self.pcs_component[0]} '{self.pcs_component[1]}' in the PCS table.")
#             # Resetting pcs_component flag
#             self.pcs_component = False

        ### ADDED ###
        global test_flag
        test_flag = "No Test"  
        # Not testing these terms
        # These terms may have a second level from which the user needs to choose

### End Class Definitions

### Running Tests

In [3]:
found = False
def run_test(mainterm_title):
    """
    Args:
        mainterm_title -> str of mainterm's 'title' value you want to test
    Returns:
        use global variable test_flag to decide if successful test
    """
    # Create generator object of all 'letter' objects
    letters = (letter for letter in index["ICD10PCS.index"]["letter"])
    # Create generator object of all 'mainterm' objects
    mainterms = (mainterms for letter in letters for mainterms in letter["mainTerm"])
    # Ask user for query mainterm
    # Search 'mainterms' for matching mainterm to user query
    for mainterm in mainterms:
        # If found matching mainterm
        if mainterm["title"] == mainterm_title:
            # Execute the mainterm
            parser = Parser()
            parser.execute_tree(mainterm, single_level_check=True)
            # Indicate we've found the term in 'mainterms' object. Used for testing purposes.
            found = True
            break
    # Determine if match found and executed
    if not found:
        # If we did not find the term in 'mainterms', Indicate failed test
        global test_flag
        test_flag = False

In [4]:
# lists used to keep track of tested terms
terms_not_tested = []
successful_tests = []
unsuccessful_tests = []
other_problem_tests = []
multi_level_terms = []

In [5]:
letters_test_list = (letter for letter in index["ICD10PCS.index"]["letter"])
mainterms_test_list = [mainterms for letter in letters_test_list for mainterms in letter["mainTerm"]]

global test_flag
test_flag = None

for test_mainterm in mainterms_test_list:
    
    # Used to see how many terms have one or more levels
    try:
        if test_mainterm["term"]:
            multi_level_terms.append(test_mainterm)
    except KeyError:
        pass
    
    # Instanciate class objects
    single_level_parser = Single_Level_Parser()
    mainterm_parser = Mainterm_Parser()

    # Run test. A global variable test_flag is used to determine if test was successful or not
    try:
        run_test(test_mainterm["title"])
    except KeyError:
        test_flag = None

    # Term has levels, we are not testing for terms with levels
    if test_flag == "No Test":
        terms_not_tested.append(test_mainterm["title"])
    # Mainterm was executed successfully
    elif test_flag == True:
        successful_tests.append(test_mainterm["title"])
    # Mainterm failed execution. Need debug
    elif test_flag == False:
        unsuccessful_tests.append(test_mainterm["title"])
    elif test_flag == None:
        other_problem_tests.append(test_mainterm["title"])
    else:
        pprint(mainterm)
        raise NameError("The value for global variable test_flag: '{test_flag}', is not recognizable.")

--------PARENT EXECUTE--------
--------execute_group_1--------
--------execute_group_1_1--------
--------PARENT EXECUTE--------
--------execute_group_1--------
--------execute_group_1_1--------
--------PARENT EXECUTE--------
--------execute_group_1--------
--------execute_group_1_1--------
--------PARENT EXECUTE--------
--------execute_group_2--------
--------execute_group_2_2--------
--------EXECUTE SINGLE LEVEL--------
--------execute_single_level_1--------
--------execute_single_level_1_2--------
--------EXECUTE SINGLE LEVEL--------
--------execute_single_level_3--------
--------PARENT EXECUTE--------
--------execute_group_1--------
--------execute_group_1_1--------
--------EXECUTE SINGLE LEVEL--------
--------execute_single_level_1--------
--------execute_single_level_1_1--------
--------EXECUTE SINGLE LEVEL--------
--------execute_single_level_5--------
--------execute_single_level_5_1--------
--------PARENT EXECUTE--------
--------execute_group_2--------
--------execute_group_2_1

--------execute_single_level_3--------
--------EXECUTE SINGLE LEVEL--------
--------execute_single_level_3--------
--------PARENT EXECUTE--------
--------execute_group_2--------
--------execute_group_2_3--------
--------PARENT EXECUTE--------
--------execute_group_2--------
--------execute_group_2_1--------
--Redirected to term 'Excision with qualifier Diagnostic'.
--------handle_pcs_table_component--------
--------PROGRESS THROUGH LEVELS--------
--------PARENT EXECUTE--------
--------execute_group_1--------
--------execute_group_1_1--------
--------PARENT EXECUTE--------
--------execute_group_1--------
--------execute_group_1_1--------
--------PARENT EXECUTE--------
--------execute_group_1--------
--------execute_group_1_1--------
--------PARENT EXECUTE--------
--------execute_group_1--------
--------execute_group_1_1--------
--------EXECUTE SINGLE LEVEL--------
--------execute_single_level_1--------
--------execute_single_level_1_2--------
--------EXECUTE SINGLE LEVEL--------
-------

--------EXECUTE SINGLE LEVEL--------
--------execute_single_level_3--------
--------EXECUTE SINGLE LEVEL--------
--------execute_single_level_3--------
--------PARENT EXECUTE--------
--------execute_group_1--------
--------execute_group_1_1--------
--------PARENT EXECUTE--------
--------execute_group_2--------
--------execute_group_2_1--------
--Redirected to term 'Fluorescence Guided Procedure'.
--Found 'Fluorescence Guided Procedure' on first attempt
--------PROGRESS THROUGH LEVELS--------
--------EXECUTE SINGLE LEVEL--------
--------execute_single_level_3--------
--------EXECUTE SINGLE LEVEL--------
--------execute_single_level_3--------
--------EXECUTE SINGLE LEVEL--------
--------execute_single_level_3--------
--------PARENT EXECUTE--------
--------execute_group_1--------
--------execute_group_1_1--------
--------PARENT EXECUTE--------
--------execute_group_1--------
--------execute_group_1_1--------
--------EXECUTE SINGLE LEVEL--------
--------execute_single_level_1--------
-----

--------execute_group_1_1--------
--------PARENT EXECUTE--------
--------execute_group_1--------
--------execute_group_1_1--------
--------EXECUTE SINGLE LEVEL--------
--------execute_single_level_3--------
--------PARENT EXECUTE--------
--------execute_group_1--------
--------execute_group_1_1--------
--------EXECUTE SINGLE LEVEL--------
--------execute_single_level_3--------
--------EXECUTE SINGLE LEVEL--------
--------execute_single_level_3--------
--------EXECUTE SINGLE LEVEL--------
--------execute_single_level_3--------
--------EXECUTE SINGLE LEVEL--------
--------execute_single_level_3--------
--------EXECUTE SINGLE LEVEL--------
--------execute_single_level_1--------
--------execute_single_level_1_1--------
--------PARENT EXECUTE--------
--------execute_group_1--------
--------execute_group_1_1--------
--------PARENT EXECUTE--------
--------execute_group_1--------
--------execute_group_1_1--------
--------PARENT EXECUTE--------
--------execute_group_5--------
--------PROGRESS T

--------PARENT EXECUTE--------
--------execute_group_1--------
--------execute_group_1_1--------
--------EXECUTE SINGLE LEVEL--------
--------execute_single_level_3--------
--------EXECUTE SINGLE LEVEL--------
--------execute_single_level_3--------
--------PARENT EXECUTE--------
--------execute_group_1--------
--------execute_group_1_1--------
--------EXECUTE SINGLE LEVEL--------
--------execute_single_level_3--------
--------PARENT EXECUTE--------
--------execute_group_1--------
--------execute_group_1_1--------
--------EXECUTE SINGLE LEVEL--------
--------execute_single_level_3--------
--------EXECUTE SINGLE LEVEL--------
--------execute_single_level_3--------
--------EXECUTE SINGLE LEVEL--------
--------execute_single_level_3--------
--------PARENT EXECUTE--------
--------execute_group_1--------
--------execute_group_1_1--------
--------EXECUTE SINGLE LEVEL--------
--------execute_single_level_3--------
--------PARENT EXECUTE--------
--------execute_group_1--------
--------execute_g

...

--------EXECUTE SINGLE LEVEL--------
--------execute_single_level_1--------
--------execute_single_level_1_2--------
--------PARENT EXECUTE--------
--------execute_group_2--------
--------execute_group_2_3--------
--------PARENT EXECUTE--------
--------execute_group_2--------
--------execute_group_2_3--------
--------EXECUTE SINGLE LEVEL--------
--------execute_single_level_3--------
--------PROGRESS THROUGH LEVELS--------
--------EXECUTE SINGLE LEVEL--------
--------execute_single_level_1--------
--------execute_single_level_1_2--------
--------EXECUTE SINGLE LEVEL--------
--------execute_single_level_1--------
--------execute_single_level_1_2--------
--------EXECUTE SINGLE LEVEL--------
--------execute_single_level_1--------
--------execute_single_level_1_2--------
--------EXECUTE SINGLE LEVEL--------
--------execute_single_level_1--------
--------execute_single_level_1_2--------
--------EXECUTE SINGLE LEVEL--------
--------execute_single_level_2--------
--------PARENT EXECUTE-------

--------PARENT EXECUTE--------
--------execute_group_1--------
--------execute_group_1_1--------
--------PARENT EXECUTE--------
--------execute_group_1--------
--------execute_group_1_1--------
--------PARENT EXECUTE--------
--------execute_group_2--------
--------execute_group_2_3--------
--------PARENT EXECUTE--------
--------execute_group_1--------
--------execute_group_1_2--------
--------PROGRESS THROUGH LEVELS--------
--------PROGRESS THROUGH LEVELS--------
--------PARENT EXECUTE--------
--------execute_group_1--------
--------execute_group_1_2--------
--------PARENT EXECUTE--------
--------execute_group_1--------
--------execute_group_1_1--------
--------PARENT EXECUTE--------
--------execute_group_1--------
--------execute_group_1_1--------
--------PARENT EXECUTE--------
--------execute_group_1--------
--------execute_group_1_1--------
--------PARENT EXECUTE--------
--------execute_group_1--------
--------execute_group_1_1--------
--------EXECUTE SINGLE LEVEL--------
--------ex

--------PARENT EXECUTE--------
--------execute_group_2--------
--------execute_group_2_2--------
--------EXECUTE SINGLE LEVEL--------
--------execute_single_level_1--------
--------execute_single_level_1_2--------
--------PARENT EXECUTE--------
--------execute_group_1--------
--------execute_group_1_1--------
--------PARENT EXECUTE--------
--------execute_group_1--------
--------execute_group_1_1--------
--------PARENT EXECUTE--------
--------execute_group_1--------
--------execute_group_1_1--------
--------PARENT EXECUTE--------
--------execute_group_1--------
--------execute_group_1_1--------
--------PARENT EXECUTE--------
--------execute_group_1--------
--------execute_group_1_1--------
--------PARENT EXECUTE--------
--------execute_group_1--------
--------execute_group_1_1--------
--------PROGRESS THROUGH LEVELS--------
--------PARENT EXECUTE--------
--------execute_group_1--------
--------execute_group_1_1--------
--------EXECUTE SINGLE LEVEL--------
--------execute_single_level_3

--------EXECUTE SINGLE LEVEL--------
--------execute_single_level_1--------
--------execute_single_level_1_2--------
--------PARENT EXECUTE--------
--------execute_group_1--------
--------execute_group_1_1--------
--------EXECUTE SINGLE LEVEL--------
--------PROGRESS THROUGH LEVELS--------
--------PARENT EXECUTE--------
--------execute_group_2--------
--------execute_group_2_2--------
--------PARENT EXECUTE--------
--------execute_group_2--------
--------execute_group_2_2--------
--------EXECUTE SINGLE LEVEL--------
--------execute_single_level_3--------
--------PROGRESS THROUGH LEVELS--------
--------PARENT EXECUTE--------
--------execute_group_1--------
--------execute_group_1_1--------
--------PARENT EXECUTE--------
--------execute_group_4--------
--------PARENT EXECUTE--------
--------execute_group_2--------
--------execute_group_2_2--------
--------PARENT EXECUTE--------
--------execute_group_1--------
--------execute_group_1_1--------
--------PARENT EXECUTE--------
--------execut

--------EXECUTE SINGLE LEVEL--------
--------execute_single_level_3--------
--------EXECUTE SINGLE LEVEL--------
--------execute_single_level_1--------
--------execute_single_level_1_2--------
--------EXECUTE SINGLE LEVEL--------
--------execute_single_level_1--------
--------execute_single_level_1_2--------
--------EXECUTE SINGLE LEVEL--------
--------execute_single_level_1--------
--------execute_single_level_1_2--------
--------PARENT EXECUTE--------
--------execute_group_2--------
--------execute_group_2_2--------
--------PARENT EXECUTE--------
--------execute_group_4--------
--------PARENT EXECUTE--------
--------execute_group_2--------
--------execute_group_2_3--------
--------PARENT EXECUTE--------
--------execute_group_2--------
--------execute_group_2_2--------
--------PARENT EXECUTE--------
--------execute_group_1--------
--------execute_group_1_1--------
--------EXECUTE SINGLE LEVEL--------
--------execute_single_level_1--------
--------execute_single_level_1_2--------
-----

### Test Results

In [6]:
print(f"Number of terms NOT tested: {len(terms_not_tested)}")

Number of terms NOT tested: 1274


In [7]:
print(f"Number of terms tested successfully: {len(successful_tests)}")

Number of terms tested successfully: 1543


In [8]:
# These are known errors
# Waiting until able to integrate with PCS Table
print(f"Number of unsuccessful tests: {len(unsuccessful_tests)}")
unsuccessful_tests

Number of unsuccessful tests: 11


['Drotrecogin alfa, infusion',
 'Eptifibatide, infusion',
 'Immunization',
 'Immunotherapy',
 'Parenteral nutrition, total',
 'Peripheral parenteral nutrition',
 'Phototherapy',
 'PPN (peripheral parenteral nutrition)',
 'Sclerotherapy, via injection of sclerosing agent',
 'Total parenteral nutrition (TPN)',
 'Vaccination']

In [9]:
print(f"Number of unsuccessful tests with problems unknown: {len(other_problem_tests)}")
assert len(other_problem_tests) == 0

Number of unsuccessful tests with problems unknown: 0


In [10]:
# Check that all terms are accounted for
total_terms_tested = len(terms_not_tested) + len(successful_tests) + len(unsuccessful_tests) + len(other_problem_tests)
assert total_terms_tested == len(mainterms_test_list), "Didn't test all terms."

### To Do:
* Test for terms not tested
* Figure out way to test all of possible combinations

In [28]:
letters = (letter for letter in index["ICD10PCS.index"]["letter"])
# Create generator object of all 'mainterm' objects
mainterms = [mainterms for letter in letters for mainterms in letter["mainTerm"]]

In [35]:
untested_terms = [term for term in mainterms if term['title'] in terms_not_tested]

In [46]:
pprint(terms_not_tested)

['Abdominoplasty',
 'Abductor hallucis muscle',
 'Ablation',
 'Abrasion',
 'Accessory cephalic vein',
 'Acetabulectomy',
 'Acetabulofemoral joint',
 'Acetabuloplasty',
 'Achilles tendon',
 'Achillotenotomy, achillotomy',
 'Acoustic Pulse Thrombolysis',
 'Acromioclavicular ligament',
 'Acromion (process)',
 'Acromionectomy',
 'Acromioplasty',
 'ACUITY(tm) Steerable Lead',
 'Acupuncture',
 'Adductor brevis muscle',
 'Adductor hallucis muscle',
 'Adductor longus muscle',
 'Adductor magnus muscle',
 'Adenoidectomy',
 'Adhesiolysis',
 'Administration',
 'Adrenalectomy',
 'Advancement',
 'Advisa (MRI)',
 'Alimentation',
 'Alteration',
 'Alveolar process of mandible',
 'Alveolectomy',
 'Alveoloplasty',
 'Alveolotomy',
 'Amputation',
 'Analog radiography',
 'Analog radiology',
 'Anastomosis',
 'Anatomical snuffbox',
 'Angiectomy',
 'Angiocardiography',
 'Angiography',
 'Angioplasty',
 'Angiorrhaphy',
 'Angiotripsy',
 'Angular vein',
 'Annular ligament',
 'Annuloplasty',
 'Anoplasty',
 'Antebra

In [40]:
mainterm = [term for term in mainterms if term["title"] == terms_not_tested[0]][0]
pprint(mainterm)

{'term': [{'_level': '1',
           'see': {'__text': 'Alteration, Abdominal Wall', 'codes': '0W0F'}},
          {'_level': '1',
           'see': {'__text': 'Repair, Abdominal Wall', 'codes': '0WQF'}},
          {'_level': '1',
           'see': {'__text': 'Supplement, Abdominal Wall', 'codes': '0WUF'}}],
 'title': 'Abdominoplasty'}


In [43]:
# When you have one level of subterms in a mainterm
for subterm in mainterm['term']:
    # Reset test flag
    global test_flag
    test_flag = None
    
    # Execute mainterm
    parser.execute_tree(subterm)
    # Successful test
    if test_flag:
        pass
    # Unsuccessful test
    elif not test_flag:
        pass
    # 'No Test' term
    elif test_flag == "No Test":
        pass
    # Unknown error
    else:
        pass

NameError: name 'parser' is not defined

### From Old Test

In [11]:
need_debug = [
    # Excision with qualifier diagnostic
#     "Core needle biopsy", # (Excision with qualifier Diagnostic). (ask coding expert how its dealt with - Connie)
#     "Punch biopsy", # (Excision with qualifier Diagnostic).
                    # Brachytherapy with qualifier Unidirectional Source
                    # Insertion with device Radioactive Element
                    # Drainage with qualifier Diagnostic
    
    # Letter casing doesn't match
#     "Diagnostic Audiology", # (Audiology, Diagnostic). Needs to be mapped to "Audiology, diagnostic"
#     "Diagnostic imaging", # (Imaging, Diagnostic). Needs to be mapped to "Imaging, diagnostic"
#     "Diagnostic radiology", # (Imaging, Diagnostic). Needs to be mapped to "Imaging, diagnostic"
#     "Radiology, diagnostic", # (Imaging, Diagnostic). Needs to be mapped to "Imaging, diagnostic" 
#     "Nuclear scintigraphy", # (Nuclear Medicine). Needs to be mapped to "Nuclear medicine"
    
    # Introduction of...
    "Drotrecogin alfa, infusion", # (Introduction of Recombinant Human-activated Protein C) (Qualifier in PCS Table)
    "Eptifibatide, infusion", # (Introduction of Platelet Inhibitor) (Substance in PCS Table)
    "Immunotherapy", # (Introduction of Immunotherapeutic Substance) (Substance in PCS Table)
    "Sclerotherapy, via injection of sclerosing agent", # (Introduction, Destructive Agent) (Substance in PCS Table)
    "Immunization", # (Introduction of Serum, Toxoid, and Vaccine) (Substance in PCS Table)
    "Vaccination", # (Introduction of Serum, Toxoid, and Vaccine) 
    "Parenteral nutrition, total", # (Introduction of Nutritional Substance) (Substance in PCS Table)
    "Peripheral parenteral nutrition", # (Introduction of Nutritional Substance)
    "PPN (peripheral parenteral nutrition)", # (Introduction of Nutritional Substance)
    "Total parenteral nutrition (TPN)", # (Introduction of Nutritional Substance)
    
    # Giving us three things
#     "Ligation, hemorrhoid", # (Occlusion, Lower Veins, Hemorrhoidal Plexus).
#     "Myocardial Bridge Release", # (Release, Artery, Coronary).
    
    #"Radiation Therapy"
]